<a href="https://colab.research.google.com/github/alzaidawi/PORTAL_code/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unzipping the dataset

In [1]:
import zipfile
import os

# Step 1: Specify the path of the zip file
zip_file_path = "/content/three_pore_typs.zip"

# Step 2: Provide the directory where you want to extract the contents
destination_folder = "/content/extracted_files"

# Step 3: Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Step 4: Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print("Extraction completed successfully.")

Extraction completed successfully.


# CNN Model 1.4 Only CNN

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# Define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Define the image data generator with data augmentation
datagen = ImageDataGenerator(
    rescale=1./360,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

# Define the image size and batch size
img_size = (360, 360)
batch_size = 32

# Prepare the training dataset
train_generator = datagen.flow_from_directory(
    '/content/extracted_files/three_pore_typs',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Prepare the validation dataset
val_generator = datagen.flow_from_directory(
    '/content/extracted_files/three_pore_typs',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10)  # Monitor 'loss' instead of 'val_loss'

# Define the learning rate scheduler callback
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model with early stopping and learning rate scheduler
history = model.fit(train_generator, validation_data=val_generator, epochs=100, callbacks=[early_stopping, lr_scheduler_callback])

# Evaluate the model
model.evaluate(val_generator)


Found 120 images belonging to 3 classes.
Found 30 images belonging to 3 classes.
Epoch 1/100
4/4 [==============================] - 56s 14s/step - loss: 5.8899 - accuracy: 0.4167 - val_loss: 4.2349 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 2/100
4/4 [==============================] - 49s 12s/step - loss: 4.2916 - accuracy: 0.6333 - val_loss: 4.2363 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 3/100
4/4 [==============================] - 50s 12s/step - loss: 4.2058 - accuracy: 0.5917 - val_loss: 3.7120 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 4/100
4/4 [==============================] - 48s 13s/step - loss: 3.5227 - accuracy: 0.6083 - val_loss: 2.9781 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 5/100
4/4 [==============================] - 50s 13s/step - loss: 3.0360 - accuracy: 0.6750 - val_loss: 2.7468 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 6/100
4/4 [==============================] - 48s 12s/step - loss: 2.6768 - accuracy: 0.7583 - val_loss: 2.5316 - val_accuracy: 0.6333 - lr: 0.0010
E

[0.5569945573806763, 1.0]

In [ ]:
# Save the trained model
model.save("/content/CNN.h5")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

def plot_training_history(history):
    plt.figure(figsize=(12, 6))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    sns.lineplot(x=range(len(history.history['accuracy'])), y=history.history['accuracy'], label='Training Accuracy')
    sns.lineplot(x=range(len(history.history['val_accuracy'])), y=history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    sns.lineplot(x=range(len(history.history['loss'])), y=history.history['loss'], label='Training Loss')
    sns.lineplot(x=range(len(history.history['val_loss'])), y=history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(history)

#TODO NEXT:
1. how to use the saved model to predict new images
2. create doc for the notebook

#**CNN for Pixel segmentation**

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

# Load and preprocess an image
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Load and preprocess a mask
def load_and_preprocess_mask(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, (256, 256))
    mask = tf.cast(mask, tf.float32) / 255.0
    return mask

# Load the dataset
def load_dataset(image_folder, mask_folder):
    image_paths = glob.glob(os.path.join(image_folder, '*.JPG'))
    mask_paths = glob.glob(os.path.join(mask_folder, '*.JPG'))

    X_train = []
    y_train = []

    for image_path in image_paths:
        mask_path = os.path.join(mask_folder, os.path.basename(image_path))

        image = load_and_preprocess_image(image_path)
        mask = load_and_preprocess_mask(mask_path)

        X_train.append(image)
        y_train.append(mask)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train

# U-Net model
def unet_model(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Middle
    conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)

    # Decoder
    up5 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv4)
    up5 = tf.keras.layers.Conv2D(256, 2, activation='relu', padding='same')(up5)
    merge5 = tf.keras.layers.concatenate([conv3, up5], axis=3)
    conv5 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(merge5)
    conv5 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv5)

    up6 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = tf.keras.layers.Conv2D(128, 2, activation='relu', padding='same')(up6)
    merge6 = tf.keras.layers.concatenate([conv2, up6], axis=3)
    conv6 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(merge6)
    conv6 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv6)

    up7 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = tf.keras.layers.Conv2D(64, 2, activation='relu', padding='same')(up7)
    merge7 = tf.keras.layers.concatenate([conv1, up7], axis=3)
    conv7 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(merge7)
    conv7 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv7)

    # Output
    output = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(conv7)

    model = tf.keras.models.Model(inputs=inputs, outputs=output)

    return model

# Define input shape (256x256 with 1 channel, as it's grayscale)
input_shape = (256, 256, 1)

# Create the U-Net model
model = unet_model(input_shape)

# Compile the model with an appropriate loss function and optimizer
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# Load and preprocess the dataset
train_image_folder = '/content/train_images'
train_mask_folder = '/content/mask_images'
X_train, y_train = load_dataset(train_image_folder, train_mask_folder)

# Train the model using the dataset
model.fit(X_train, y_train, batch_size=16, epochs=10)

# After training, you can use the model for inference

# Example: Performing inference on a test image
test_image_path = '/content/test_images/144.JPG'
test_image = load_and_preprocess_image(test_image_path)
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))

# Display the results
plt.subplot(1, 2, 1)
plt.imshow(np.squeeze(test_image), cmap='gray')
plt.title('Test Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.squeeze(predicted_mask), cmap='gray')
plt.title('Predicted Mask')
plt.axis('off')

plt.show()
